Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your name below:

In [1]:
let name = "Sathvick reddy beeravalli"
let rollno = "cs19b010"

val name : string = "Sathvick reddy beeravalli"


val rollno : string = "cs19b010"


## Important notes about grading:

1. **Compiler errors:** All code you submit must compile. Programs that do not compile will probably receive an automatic zero. If you are having trouble getting your assignment to compile, please visit consulting hours. If you run out of time, it is better to comment out the parts that do not compile, than hand in a more complete file that does not compile.
2. **Late assignments:** Please carefully review the course website's policy on late assignments, as all assignments handed in after the deadline will be considered late. Verify on moodle that you have submitted the correct version, before the deadline. Submitting the incorrect version before the deadline and realizing that you have done so after the deadline will be counted as a late submission.

## A Prolog interpreter in OCaml

In this assignment, you will implement a Prolog interpreter in OCaml with support for backtracking (recover from bad choices) and choice points (produce multiple results).

We will start with the definition of terms, predicates, clauses, goals and programs.

In [2]:
type constant = Constant of string
type variable = Variable of string
type func = Function of string * term list
and term = C of constant | V of variable | F of func

type predicate = Predicate of string * term list

type head = predicate
type body = predicate list

type clause = Fact of head | Rule of head * body

type goal = predicate list

type program = clause list

type constant = Constant of string


type variable = Variable of string


type func = Function of string * term list
and term = C of constant | V of variable | F of func


type predicate = Predicate of string * term list


type head = predicate


type body = predicate list


type clause = Fact of head | Rule of head * body


type goal = predicate list


type program = clause list


The following stringification functions should help you debug the interpreter. 

In [3]:
let rec string_of_f_list f tl =
  let _, s = List.fold_left (fun (first, s) t -> 
    let prefix = if first then "" else s ^ ", " in
    false, prefix ^ (f t)) (true,"") tl
  in 
  s

let rec string_of_term t =
  match t with
  | C (Constant c) -> c
  | V (Variable v) -> v
  | F (Function (f,tl)) -> 
      f ^ "(" ^ (string_of_f_list string_of_term tl) ^ ")"

let string_of_predicate (Predicate (p,tl)) =
  p ^ "(" ^ (string_of_f_list string_of_term tl) ^")"
  
let string_of_predicate_list pl =
  string_of_f_list string_of_predicate pl
  
let string_of_goal g =
  "?- " ^ (string_of_predicate_list g)

let string_of_clause c =
  match c with
  | Fact f -> string_of_predicate f ^ "."
  | Rule (h,b) -> string_of_predicate h ^ " :- " ^ (string_of_predicate_list b) ^ "."

let string_of_program p =
  List.fold_left (fun acc c -> acc ^ (string_of_clause c) ^ "\n") "" p

val string_of_f_list : ('a -> string) -> 'a list -> string = <fun>


val string_of_term : term -> string = <fun>


val string_of_predicate : predicate -> string = <fun>


val string_of_predicate_list : predicate list -> string = <fun>


val string_of_goal : predicate list -> string = <fun>


val string_of_clause : clause -> string = <fun>


val string_of_program : clause list -> string = <fun>


Below are more helper functions:

In [4]:
let var v = Variable v
let var_t v = V (var v)
let const c = Constant c
let const_t c = C (Constant c)
let func f l = Function (f,l)
let func_t f l = F (func f l)
let pred p l = Predicate (p,l)
let fact f = Fact f
let rule h b = Rule (h,b)

val var : string -> variable = <fun>


val var_t : string -> term = <fun>


val const : string -> constant = <fun>


val const_t : string -> term = <fun>


val func : string -> term list -> func = <fun>


val func_t : string -> term list -> term = <fun>


val pred : string -> term list -> predicate = <fun>


val fact : head -> clause = <fun>


val rule : head -> body -> clause = <fun>


## Problem 1

In this problem, you will implement the function:

```ocaml
occurs_check : variable -> term -> bool
```

`occurs_check v t` returns `true` if the variable `v` occurs in `t`. 

In [5]:
let rec check v lis=
match lis with
|[]->false
|hd::tl-> match hd with
          | C (Constant c) -> check v tl
          | V (Variable var) -> if(v=(Variable var)) then true else check v tl
          | F (Function (f,tl1)) ->  check v tl1
 
 
let rec occurs_check v t = 
match t with
 | C (Constant c) -> false
 | V (Variable var) -> if(v=(Variable var)) then true else false
 | F (Function (f,tl)) -> check v tl 

val check : variable -> term list -> bool = <fun>


val occurs_check : variable -> term -> bool = <fun>


In [6]:
(* 5 points *)
assert (occurs_check (var "X") (var_t "X"));
assert (not (occurs_check (var "X") (var_t "Y")));
assert (occurs_check (var "X") (func_t "f" [var_t "X"]))

- : unit = ()


## Problem 2

Implement the following functions which return the variables contained in them:

```ocaml
variables_of_term      : term -> VarSet.t
variables_of_predicate : predicate -> VarSet.t
variables_of_clause    : clause -> VarSet.t
```

In [7]:
module VarSet = Set.Make(struct type t = variable let compare = Pervasives.compare end)
(* API Docs for Set : https://caml.inria.fr/pub/docs/manual-ocaml/libref/Set.S.html *)


let rec check lis acc1=
match lis with
|[]->acc1
|hd::tl-> match hd with
          | C (Constant c) -> check  tl acc1
          | V (Variable var) -> check  tl (VarSet.add (Variable var) acc1)
          | F (Function (f,tl1)) ->  check  tl1 acc1
let check  lis= let acc1=VarSet.empty in
 check lis acc1

let rec variables_of_term t acc= 
    match t with
 | C (Constant c) -> acc
 | V (Variable var) -> VarSet.add (Variable var) acc
 | F (Function (f,tl)) -> check  tl 
let variables_of_term t= let acc=VarSet.empty in
 variables_of_term t acc
  

let variables_of_predicate (Predicate (_, l)) = 
 check l 

let rec predlist lis acc1=
match lis with
|[]->acc1
|h::t->predlist t (VarSet.union (variables_of_predicate h) acc1)
  
let variables_of_clause c =
match c with
  | Fact f -> variables_of_predicate f
  | Rule (h,b) ->predlist b (variables_of_predicate h)   


File "[7]", line 1, characters 64-82:
1 | module VarSet = Set.Make(struct type t = variable let compare = Pervasives.compare end)
                                                                    ^^^^^^^^^^^^^^^^^^
Alert deprecated: module Stdlib.Pervasives
Use Stdlib instead.

If you need to stay compatible with OCaml < 4.07, you can use the 
stdlib-shims library: https://github.com/ocaml/stdlib-shims


module VarSet :
  sig
    type elt = variable
    type t
    val empty : t
    val is_empty : t -> bool
    val mem : elt -> t -> bool
    val add : elt -> t -> t
    val singleton : elt -> t
    val remove : elt -> t -> t
    val union : t -> t -> t
    val inter : t -> t -> t
    val disjoint : t -> t -> bool
    val diff : t -> t -> t
    val compare : t -> t -> int
    val equal : t -> t -> bool
    val subset : t -> t -> bool
    val iter : (elt -> unit) -> t -> unit
    val map : (elt -> elt) -> t -> t
    val fold : (elt -> 'a -> 'a) -> t -> 'a -> 'a
    val for_all : (elt -> bool) -> t -> bool
    val exists : (elt -> bool) -> t -> bool
    val filter : (elt -> bool) -> t -> t
    val partition : (elt -> bool) -> t -> t * t
    val cardinal : t -> int
    val elements : t -> elt list
    val min_elt : t -> elt
    val min_elt_opt : t -> elt option
    val max_elt : t -> elt
    val max_elt_opt : t -> elt option
    val choose : t -> elt
    val choose_opt : t -> elt option
    

val check : term list -> VarSet.t -> VarSet.t = <fun>


val check : term list -> VarSet.t = <fun>


val variables_of_term : term -> VarSet.t -> VarSet.t = <fun>


val variables_of_term : term -> VarSet.t = <fun>


val variables_of_predicate : predicate -> VarSet.t = <fun>


val predlist : predicate list -> VarSet.t -> VarSet.t = <fun>


val variables_of_clause : clause -> VarSet.t = <fun>


In [8]:
(* 5 points *)
assert (variables_of_term (func_t "f" [var_t "X"; var_t "Y"; const_t "a"]) = 
        VarSet.of_list [var "X"; var "Y"]);
assert (variables_of_predicate (pred "p" [var_t "X"; var_t "Y"; const_t "a"]) = 
        VarSet.of_list [var "X"; var "Y"]);
assert (variables_of_clause (fact @@ pred "p" [var_t "X"; var_t "Y"; const_t "a"]) = 
        VarSet.of_list [var "X"; var "Y"]);
assert (variables_of_clause (rule (pred "p" [var_t "X"; var_t "Y"; const_t "a"]) [pred "q" [const_t "a"; const_t "b"; const_t "a"]])= 
        VarSet.of_list [var "X"; var "Y"])

- : unit = ()


## Problem 3

The value of type `term Substitution.t` is a OCaml map whose key is of type `variable` and value is of type `term`. In other words, it is map from variables to terms. Implement substitution functions that takes a `term Substitution.t` and uses that to perform the substitution:

```ocaml
substitute_in_term : term Substitution.t -> term -> term
substitute_in_predicate : term Substitution.t -> predicate -> predicate
substitute_in_clause : term Substitution.t -> clause -> clause
```

In [9]:
(*module Substitution = Map.Make(struct type t = variable let compare = Pervasives.compare end)
(* See API docs for OCaml Map: https://caml.inria.fr/pub/docs/manual-ocaml/libref/Map.S.html *)

let string_of_substitution s =
  "{" ^ (Substitution.fold (fun (Variable v) t s -> s ^ "; " ^ v ^ " -> " ^ (string_of_term t)) s "") ^ "}"


let rec check s lis =
match lis with
|[]->[]
|hd::tl-> match hd with
          | C (Constant c) -> hd::(check s tl) 
          | V (Variable var) -> if (Substitution.mem (Variable var) s) then (Substitution.find (Variable var) s)::(check s tl ) else hd::(check s tl)
          | F (Function (f,tl1)) -> (F (Function(f,(check  s tl1))))::(check s tl)


let rec substitute_in_term s t =
  match t with
 | C (Constant c) -> t
 | V (Variable var) ->if (Substitution.mem (Variable var) s) then (Substitution.find (Variable var) s) else t
 | F (Function (f,tl)) ->F (Function(f,(check s tl)))   



let rec substitute_in_predicate s (Predicate (p,l)) =
 Predicate(p,check s l)


let rec predlist s lis acc1=
match lis with
|[]->acc1
|h::t->predlist s t ((substitute_in_predicate s h)::acc1)

let substitute_in_clause s c =
match c with
  | Fact f ->Fact (substitute_in_predicate s f) 
  | Rule (h,b) ->Rule((substitute_in_predicate s h),(predlist s b []))*)
  
  
 module Substitution = Map.Make(struct type t = variable let compare = Pervasives.compare end)
(* See API docs for OCaml Map: https://caml.inria.fr/pub/docs/manual-ocaml/libref/Map.S.html *)

let string_of_substitution s =
  "{" ^ (Substitution.fold (fun (Variable v) t s -> s ^ "; " ^ v ^ " -> " ^ (string_of_term t)) s "") ^ "}"






let rec check s lis =
match lis with
|[]->[]
|hd::tl-> match hd with
          | C (Constant c) -> hd::(check s tl) 
          | V (Variable var) -> if (Substitution.mem (Variable var) s) then (Substitution.find (Variable var) s)::(check s tl ) else hd::(check s tl)
          | F (Function (f,tl1)) -> (F (Function(f,(check  s tl1))))::(check s tl)


let rec substitute_in_term s t =
  match t with
 | C (Constant c) -> t
 | V (Variable var) ->if (Substitution.mem (Variable var) s) then (Substitution.find (Variable var) s) else t
 | F (Function (f,tl)) ->F (Function(f,(check s tl)))   


let rec substitute_in_predicate s (Predicate (p,l)) =
 Predicate(p,check s l)



(*let rec substitute_in_predicate s (Predicate (p,l)) =
 Predicate(p,List.fold_left(fun acc x-> acc@[substitute_in_term s x]) [] l)
*)



let sub s pl = List.fold_left(fun acc x-> acc@[substitute_in_predicate s x]) [] pl


let substitute_in_clause s c =
match c with
  | Fact f ->Fact (substitute_in_predicate s f) 
  | Rule (h,b) ->Rule ((substitute_in_predicate s h),(List.fold_left(fun acc x-> acc@[substitute_in_predicate s x])[] b)) 
  

File "[9]", line 40, characters 71-89:
40 |  module Substitution = Map.Make(struct type t = variable let compare = Pervasives.compare end)
                                                                            ^^^^^^^^^^^^^^^^^^
Alert deprecated: module Stdlib.Pervasives
Use Stdlib instead.

If you need to stay compatible with OCaml < 4.07, you can use the 
stdlib-shims library: https://github.com/ocaml/stdlib-shims


module Substitution :
  sig
    type key = variable
    type +'a t
    val empty : 'a t
    val is_empty : 'a t -> bool
    val mem : key -> 'a t -> bool
    val add : key -> 'a -> 'a t -> 'a t
    val update : key -> ('a option -> 'a option) -> 'a t -> 'a t
    val singleton : key -> 'a -> 'a t
    val remove : key -> 'a t -> 'a t
    val merge :
      (key -> 'a option -> 'b option -> 'c option) -> 'a t -> 'b t -> 'c t
    val union : (key -> 'a -> 'a -> 'a option) -> 'a t -> 'a t -> 'a t
    val compare : ('a -> 'a -> int) -> 'a t -> 'a t -> int
    val equal : ('a -> 'a -> bool) -> 'a t -> 'a t -> bool
    val iter : (key -> 'a -> unit) -> 'a t -> unit
    val fold : (key -> 'a -> 'b -> 'b) -> 'a t -> 'b -> 'b
    val for_all : (key -> 'a -> bool) -> 'a t -> bool
    val exists : (key -> 'a -> bool) -> 'a t -> bool
    val filter : (key -> 'a -> bool) -> 'a t -> 'a t
    val partition : (key -> 'a -> bool) -> 'a t -> 'a t * 'a t
    val cardinal : 'a t -> int
    val bindings : 'a 

val string_of_substitution : term Substitution.t -> string = <fun>


val check : term Substitution.t -> term list -> term list = <fun>


val substitute_in_term : term Substitution.t -> term -> term = <fun>


val substitute_in_predicate : term Substitution.t -> predicate -> predicate =
  <fun>


val sub : term Substitution.t -> predicate list -> predicate list = <fun>


val substitute_in_clause : term Substitution.t -> clause -> clause = <fun>


In [10]:
let s = 
  let open Substitution in 
  add (var "Y") (const_t "0") (add (var "X") (var_t "Y") empty)
in
assert (substitute_in_term s (func_t "f" [var_t "X"; var_t "Y"; const_t "a"]) = 
        func_t "f" [var_t "Y"; const_t "0"; const_t "a"]);
assert (substitute_in_predicate s (pred "p" [var_t "X"; var_t "Y"; const_t "a"]) = 
        pred "p" [var_t "Y"; const_t "0"; const_t "a"]);

- : unit = ()


In [11]:
(* 10 points *)
let s = 
  let open Substitution in 
  add (var "Y") (const_t "0") (add (var "X") (var_t "Y") empty)
in
assert (substitute_in_term s (func_t "f" [var_t "X"; var_t "Y"; const_t "a"]) = 
        func_t "f" [var_t "Y"; const_t "0"; const_t "a"]);
assert (substitute_in_predicate s (pred "p" [var_t "X"; var_t "Y"; const_t "a"]) = 
        pred "p" [var_t "Y"; const_t "0"; const_t "a"]);
assert (substitute_in_clause s (fact @@ pred "p" [var_t "X"; var_t "Y"; const_t "a"]) = 
        (fact @@ pred "p" [var_t "Y"; const_t "0"; const_t "a"]));
assert (substitute_in_clause s (rule (pred "p" [var_t "X"; var_t "Y"; const_t "a"]) [pred "q" [const_t "a"; const_t "b"; const_t "a"]]) = 
        (rule (pred "p" [var_t "Y"; const_t "0"; const_t "a"]) [pred "q" [const_t "a"; const_t "b"; const_t "a"]]))

- : unit = ()


We define a function `freshen` that given a clause, returns a clause where are the variables have been renamed with fresh variables. 

In [12]:
let counter = ref 0
let fresh () = 
  let c = !counter in
  counter := !counter + 1;
  V (Variable ("_G" ^ string_of_int c))

let freshen c =
  let vars = variables_of_clause c in
  let s = VarSet.fold (fun v s -> Substitution.add v (fresh()) s) vars Substitution.empty in
  substitute_in_clause s c

val counter : int ref = {contents = 0}


val fresh : unit -> term = <fun>


val freshen : clause -> clause = <fun>


For example, 

In [13]:
let c = (rule (pred "p" [var_t "X"; var_t "Y"; const_t "a"]) [pred "q" [var_t "X"; const_t "b"; const_t "a"]])
let _ = string_of_clause c
let _ = string_of_clause (freshen c)

val c : clause =
  Rule
   (Predicate ("p", [V (Variable "X"); V (Variable "Y"); C (Constant "a")]),
   [Predicate ("q", [V (Variable "X"); C (Constant "b"); C (Constant "a")])])


- : string = "p(X, Y, a) :- q(X, b, a)."


- : string = "p(_G0, _G1, a) :- q(_G0, b, a)."


## Problem 4

Implement the function:

```ocaml
mgu : term -> term -> term Substitution.t
```

returns the most general unifier of the given terms. The function should raise the exception `Not_unfifiable`, if the given terms are not unifiable. 

Using `mgu`, implement the function:

```ocaml
mgu_predicate : predicate -> predicate -> term Substitution.t
```

which returns the most general unifier of the given predicates. 

In [14]:
exception Not_unifiable


let rec add h key valu=
match h with
|[]->[(key,valu)]
|(a,b)::t-> if(b=V(key)) then (a,valu)::(add t key valu)  else (a,b)::(add t key valu)

 

let rec ltomap lis =
match lis with
|[]->Substitution.empty
|(a,b)::t->Substitution.add a b (ltomap t)

let rec mgu1 t1 t2 acc= 
  
 let t1=substitute_in_term acc t1 in
 let t2=substitute_in_term acc t2 in
  
 match t1 with 
 | C (Constant c) -> (match t2 with 
                     | C (Constant c1) -> if c1=c then acc else raise Not_unifiable
                     | V (Variable v1) -> ltomap (add (Substitution.bindings acc) (var v1) (const_t c))
                     | F (Function (f,tl1)) -> raise Not_unifiable)
  
 | V (Variable v) -> (match t2 with 
                     | C (Constant c1) -> ltomap (add (Substitution.bindings acc) (var v) (const_t c1))
                     | V (Variable v1) -> if v1=v then acc else ltomap (add (Substitution.bindings acc) (var v) (var_t v1))
                     | F (Function (f,tl1)) -> if(occurs_check (var v) (func_t f tl1 ) ) then raise Not_unifiable else ltomap (add (Substitution.bindings acc) (var v) t2)
                      )
 | F (Function (f,tl)) -> (
                           match t2 with 
                           | C (Constant c1) ->  raise Not_unifiable
                           | V (Variable v1) ->  if(occurs_check (var v1) (func_t f tl ) ) then raise Not_unifiable else ltomap (add (Substitution.bindings acc) (var v1) t1)
                           | F (Function (f1,tl1)) -> if f=f1 then (if (List.length tl=List.length tl1) then ( List.fold_left(fun acc (x,y)->mgu1 x y acc) acc (List.combine tl tl1) ) else raise Not_unifiable) else raise Not_unifiable
                            
                           );;
  
  
let mgu t1 t2  = mgu1 t1 t2 (Substitution.empty) ;;               


let rec fpred x lis1 lis2=
match lis1 with
|[]->x
|h::t-> (match lis2 with 
         |[]->x
         |h1::t1->fpred (mgu1 h h1 x) t t1
         )

let mgu_predicate (Predicate (p1,l1)) (Predicate (p2,l2)) =
if (p1=p2) then (if((List.length l1=List.length l2)) then fpred (Substitution.empty) l1 l2 else raise Not_unifiable) else raise Not_unifiable


exception Not_unifiable


val add :
  (variable * term) list -> variable -> term -> (variable * term) list =
  <fun>


val ltomap : (Substitution.key * 'a) list -> 'a Substitution.t = <fun>


val mgu1 : term -> term -> term Substitution.t -> term Substitution.t = <fun>


val mgu : term -> term -> term Substitution.t = <fun>


val fpred :
  term Substitution.t -> term list -> term list -> term Substitution.t =
  <fun>


val mgu_predicate : predicate -> predicate -> term Substitution.t = <fun>


In [15]:
(* 15 points *)
assert (string_of_substitution (mgu (var_t "X") (var_t "Y")) = "{; X -> Y}" ||
        string_of_substitution (mgu (var_t "X") (var_t "Y")) = "{; Y -> X}");
assert (string_of_substitution (mgu (var_t "Y") (var_t "X")) = "{; X -> Y}" ||
        string_of_substitution (mgu (var_t "Y") (var_t "X")) = "{; Y -> X}");
assert (mgu (var_t "Y") (var_t "Y") = Substitution.empty);
assert (mgu (const_t "0") (const_t "0") = Substitution.empty);
assert (mgu (const_t "0") (var_t "Y") = Substitution.singleton (var "Y") (const_t "0"));
assert (
  match mgu (const_t "0") (const_t "1") with
  | _ -> false 
  | exception Not_unifiable -> true);
assert (
  match mgu (func_t "f" [const_t "0"]) (func_t "g" [const_t "1"]) with
  | _ -> false 
  | exception Not_unifiable -> true);
assert (mgu (func_t "f" [var_t "X"]) (func_t "f" [var_t "Y"]) = Substitution.singleton (var "X") (var_t "Y") ||
        mgu (func_t "f" [var_t "X"]) (func_t "f" [var_t "Y"]) = Substitution.singleton (var "Y") (var_t "X"))

- : unit = ()


In [16]:
(* 15 points *)
let t1 = F (Function("f", [V (Variable "X"); V (Variable "Y"); V (Variable "Y")])) in 
let t2 = F (Function("f", [V (Variable "Y"); V (Variable "Z"); V (Variable "a")])) in
let u = mgu t1 t2 in
assert (Substitution.cardinal u = 3);
assert (string_of_substitution u = "{; X -> a; Y -> a; Z -> a}");


- : unit = ()


## Problem 5

Implement the function:

```ocaml
query : ?limit:int -> clause list -> predicate list -> predicate list list 
```

where 

* the first optional argument is the `limit` i.e, maximum number of results to be returned. By default, the limit is set to 10 (see code below).
* the second argument is the `program` which is a list of clauses.
* the third argument is the `goal` which is a list of predicates.

The function returns a list of predicate lists (`results`). Each of these results is a instance of the original goal and is a logical consequence of the program. If the given goal is not a logical consequence of the program, then the result is an empty list. See tests cases for expected results.

For the rule and goal order, choose what Prolog does; choose the left-most subgoal for goal order and first rule in the order in which the rules appear in the program for the rule order. 

Hint: Implement a purely functional recursive solution. The backtracking and choice points naturally fall out of the implementation. The reference solution is 17 lines long. 

In [17]:
let sub s pl = List.fold_left(fun acc x-> acc@[substitute_in_predicate s x]) [] pl


let query ?(limit=10) program goal = 
let rec help (goa) (pro) (res) (ans) = if(List.length ans < limit) then
List.fold_left(fun acc obj -> let fx = freshen obj in match res with
|[]->acc
|_-> begin match fx with
        | Fact f -> begin match (mgu_predicate (List.hd res) f) with
                      | exception Not_unifiable -> acc
                      |_ -> if(List.length (List.tl res) = 0) then acc@[ sub (mgu_predicate (List.hd res) f) goa]
                            else help  (sub (mgu_predicate (List.hd res) f) goa) pro (sub  (mgu_predicate (List.hd res) f)(List.tl res)) acc
                     end;
        | Rule(h,b) -> let n = b@(List.tl res) in begin match (mgu_predicate (List.hd res) h ) with
                                                    |_-> if(List.length n = 0) then acc@[sub (mgu_predicate (List.hd res) h ) goa ]
                                                        else help (sub (mgu_predicate (List.hd res) h ) goa ) (pro) (sub (mgu_predicate (List.hd res) h ) n) acc
                                                    | exception Not_unifiable -> acc
                                                   end;
       end;                                            
) (ans) (pro)

else ans 
in help (goal) (program) (goal) ([]) 

val sub : term Substitution.t -> predicate list -> predicate list = <fun>


val query :
  ?limit:int -> clause list -> predicate list -> predicate list list = <fun>


In [18]:
(* 10 points *)
let p = [fact @@ pred "f" [const_t "a"; const_t "b"]] in
let g = [pred "f" [const_t "a"; const_t "b"]] in
let g' = match query p g with [v] -> v | _ -> failwith "error" in
assert (g' = g)

- : unit = ()


In [19]:
(* 10 points *)
let p = [fact @@ pred "f" [const_t "a"; const_t "b"]] in
let g = [pred "f" [const_t "a"; const_t "c"]] in
assert (query p g = [])

- : unit = ()


In [20]:
(* 10 points *)
let p = [fact @@ pred "f" [const_t "a"; const_t "b"]] in
let g = [pred "f" [var_t "X"; const_t "b"]] in
let g' = match query p g with [v] -> v | _ -> failwith "error" in
assert (g' = [pred "f" [const_t "a"; const_t "b"]]);

let g = [pred "f" [var_t "X"; var_t "Y"]] in
let g' = match query p g with [v] -> v | _ -> failwith "error" in
assert (g' = [pred "f" [const_t "a"; const_t "b"]])

- : unit = ()


In [21]:
(* 10 points *)
let ancestor x y = pred "ancestor" [x;y] in
let father x y = pred "father" [x;y] in
let father_consts x y =  father (C (Constant x)) (C (Constant y)) in

let f1 = Fact (father_consts "rickard" "ned") in
let f2 = Fact (father_consts "ned" "robb") in
let r1 = Rule (ancestor (var_t "X") (var_t "Y"), [father (var_t "X") (var_t "Y")]) in
let r2 = Rule (ancestor (var_t "X") (var_t "Y"), [father (var_t "X") (var_t "Z"); ancestor (var_t "Z") (var_t "Y")]) in

let p = [f1;f2;r1;r2] in
let g = [ancestor (const_t "rickard") (const_t "ned")] in
let g' = match query p g with [v] -> v | _ -> failwith "error" in
assert (g' = g)

- : unit = ()


In [22]:
(* 20 points *)
(* Tests backtracking *)
let ancestor x y = pred "ancestor" [x;y] in
let father x y = pred "father" [x;y] in
let father_consts x y =  father (C (Constant x)) (C (Constant y)) in

let f1 = Fact (father_consts "rickard" "ned") in
let f2 = Fact (father_consts "ned" "robb") in
let r1 = Rule (ancestor (var_t "X") (var_t "Y"), [father (var_t "X") (var_t "Y")]) in
let r2 = Rule (ancestor (var_t "X") (var_t "Y"), [father (var_t "X") (var_t "Z"); ancestor (var_t "Z") (var_t "Y")]) in

let p = [f1;f2;r1;r2] in

let g = [ancestor (const_t "rickard") (const_t "robb")] in
let g' = match query p g with [v] -> v | _ -> failwith "error" in
assert (g' = g)

- : unit = ()


In [23]:
(* 15 points *)
(* Tests choice points *)
let ancestor x y = pred "ancestor" [x;y] in
let father x y = pred "father" [x;y] in
let father_consts x y =  father (C (Constant x)) (C (Constant y)) in

let f1 = Fact (father_consts "rickard" "ned") in
let f2 = Fact (father_consts "ned" "robb") in
let r1 = Rule (ancestor (var_t "X") (var_t "Y"), [father (var_t "X") (var_t "Y")]) in
let r2 = Rule (ancestor (var_t "X") (var_t "Y"), [father (var_t "X") (var_t "Z"); ancestor (var_t "Z") (var_t "Y")]) in

let p = [f1;f2;r1;r2] in

let g = [ancestor (var_t "X") (const_t "robb")] in
let g1,g2 = match query p g with [v1;v2] -> v1,v2 | _ -> failwith "error" in
assert (g1 = [ancestor (const_t "ned") (const_t "robb")]);
assert (g2 = [ancestor (const_t "rickard") (const_t "robb")])

- : unit = ()


In [24]:
(* 15 points *)
(* Tests choice points *)
let nil = const_t "nil" in
let cons h t = func_t "cons" [h;t] in
let append x y z = pred "append" [x;y;z] in
let c1 = fact @@ append nil (var_t "Q") (var_t "Q") in
let c2 = rule (append (cons (var_t "H") (var_t "P")) (var_t "Q") (cons (var_t "H") (var_t "R")))
              [append (var_t "P") (var_t "Q") (var_t "R")] in
let p = [c1;c2] in

let g = [append (var_t "X") (var_t "Y") (cons (const_t "1") (cons (const_t "2") (cons (const_t "3") nil)))] in

let g' = query p g in
assert (List.length g' = 4)

- : unit = ()
